# 1. Getting Started

A verse scenario is defined by a map, a set of agents and, if there exist multiple agents, a sensor. 

In this section, we are going to look at how to create a simple scenario in Verse. In this scenario we will have only one car following a straight lane and will brake after reaching a certain position. 

## 1.1 Instantiate map
The map of a scenario specifies the tracks that the agents can follow. In this example, we will only look at a simple map we one straight lane aligned with the x-axis with type <code>T0</code>, which are also referred as track mode in later sections. We will talk about how to create more interesting maps in later sections

In [ ]:
from tutorial_map import M1

map1 = M1()

## 1.2 Creating agent
To create such a scenario in Verse, we first need to create an agent for Verse. An agent in Verse is defined by a set of tactical modes, a decision logic to determine the transition between tactical modes, and a flow function that defines continuous evolution. The agent's tactical mode and decision logic are provided as python code strings and the flow function is provided as a python function. 

The tactical mode of the agents corresponds to an agent's decision. For example, in this car braking example, the tactical mode for the agent can be <code>Normal</code> and <code>Brake</code>. The decision logic also need to know the available track modes from the map. The tactical modes and track modes are provided as <code>Enums</code> to Verse.  

In [ ]:
from enum import Enum, auto

class AgentMode(Enum):
    Normal = auto()
    Brake = auto()
    
class TrackMode(Enum):
    T0 = auto()

We also require the user to provide the continuous and discrete variables of the agents together with the decision logic. The variables are provided inside class with name <code>State</code>. Variables end with <code>_mode</code> will be identify by verse as discrete variables. In the example below, <code>agent_mode</code> and <code>track_mode</code> are the discrete variables and <code>x</code>, <code>y</code>, <code>theta</code>, <code>v</code> are the continuous variables. The type hints for the discrete variables are necessary to associate discrete variables with the tactical modes and lane modes defined above

In [ ]:
class State:
    x:float
    y:float
    theta:float
    v:float
    agent_mode:AgentMode 
    track_mode:TrackMode 

    def __init__(self, x, y, theta, v, agent_mode: AgentMode, track_mode: TrackMode):
        pass

The decision logic describe for an agent takes as input its current state and the (observable) states of the other agents if there's any, and updates the tactical mode of the ego agent. In this example, the decision logic is traight forward: When the x position of the car is above certain threshold, the car will starts to brake. This's no other agents in this scenario. The decision logic of the agent can be written in an expressive subset of Python inside function <code>decisionLogic</code>. 

In [ ]:
import copy
def decisionLogic(ego:State, track_map):
    output = copy.deepcopy(ego)
    if ego.agent_mode == AgentMode.Normal:
        if ego.x > 10:
            output.agent_mode = AgentMode.Brake
    return output

We incoperate the above definition of tactical modes and decision logic into code strings and combine it with an imported agent flow, we can then obtain the agent for this sceanrio. 

In [ ]:
from tutorial_agent import Agent1
car = Agent1('car', file_name="dl_sec1.py")

## 1.3 Creating scenario
With the agent and map defined, we can now define the scenario.

In [ ]:
from verse.scenario import Scenario
scenario = Scenario()

We can set the initial condition of the agent and add the agent 

In [ ]:
car.set_initial([[0,-0.5,0,2],[1,0.5,0,2]], (AgentMode.Normal, TrackMode.T0))
scenario.add_agent(car)

and set the map for the sceanrio

In [ ]:
scenario.set_map(map1)

Since we only have one agent in the scenario, we don't need to specify a sensor. 

We can then compute simulation traces or reachable states for the scenario

In [ ]:
traces_simu = scenario.simulate(10, 0.01)
traces_veri = scenario.verify(10, 0.01)

We can visualize the results using functions provided with Verse

In [ ]:
import plotly.graph_objects as go
from verse.plotter.plotter2D import *

fig = go.Figure()
fig = simulation_tree(traces_simu, None, fig, 0, 1, [0, 1], 'lines', 'trace')
fig.show()

fig = go.Figure()
fig = reachtube_tree(traces_veri, None, fig, 0, 1, [0, 1], 'lines', 'trace')
fig.show()

# 2. Adding multiple agents

In the previous section, we played around with a scenario with a single vehicle. In this example, we will create a scenario with multiple agents and explore the potential of Verse to handle multi-agent scenarios. 

In this example, we will look at a two car braking example. Two cars will be running in same direction on the same lane with the front car running slower than the later car. The later car will brake and stop when it gets in 5m from the front car.  

We will first setup a new scenario and add the map.

In [ ]:
from verse.scenario import Scenario
from tutorial_map import M1
scenario = Scenario()
scenario.set_map(M1())

We will use the same tactical mode, track mode and state definition as last section.

In [ ]:
from enum import Enum, auto

class AgentMode(Enum):
    Normal = auto()
    Brake = auto()
    
class TrackMode(Enum):
    T0 = auto()
    
class State:
    x:float
    y:float
    theta:float
    v:float
    agent_mode:AgentMode 
    track_mode:TrackMode 

    def __init__(self, x, y, theta, v, agent_mode: AgentMode, track_mode: TrackMode):
        pass

Verse's decision logic support python <code>any</code> and <code>all</code> functions, which allows the decision logic to quantify over other agents in the scenario. This enable user to easily create multi-agent scenario. In this case, the decision logic will take an additional argument <code>others</code>, which provides the states for other agents. Notice the type hint for both <code>ego</code> and <code>others</code>. The updated agent's decision logic looks like following.

In [ ]:
from typing import List
import copy
def decisionLogic(ego:State, others: List[State], track_map):
    output = copy.deepcopy(ego)
    if ego.agent_mode == AgentMode.Normal:
        if any(other.x-ego.x< 8 and other.x-ego.x>0 for other in others):
            output.agent_mode = AgentMode.Brake
    return output

With the updated decision logic, we can now spawn the two agents with their initial conditions. 

In [ ]:
from tutorial_agent import Agent1
car1 = Agent1('car1', file_name="dl_sec2.py")
car1.set_initial([[0,-0.5,0,2],[1,0.5,0,2]], (AgentMode.Normal, TrackMode.T0))
car2 = Agent1('car2', file_name="dl_sec2.py")
car2.set_initial([[15,-0.5,0,1],[16,0.5,0,1]], (AgentMode.Normal, TrackMode.T0))

We can then add both agents to the scenario

In [ ]:
scenario.add_agent(car1)
scenario.add_agent(car2)

With multiple agents in the scenario, we now need to add the sensor. The sensor defines how an agent is visible to other agents. In this example, we will use the default sensor function, which allows all agents to see all variables of other agents. 

In [ ]:
from tutorial_sensor import DefaultSensor
scenario.set_sensor(DefaultSensor())

With the scenario fully constructed, we can now simulte or verify the scenario using the simulate/verify function provided by Verse. 

In [ ]:
traces_simu = scenario.simulate(10, 0.01)
traces_veri = scenario.verify(10, 0.01)

We can visualize the results using functions provided with Verse

In [ ]:
import plotly.graph_objects as go
from verse.plotter.plotter2D import *

fig = go.Figure()
fig = simulation_tree(traces_simu, None, fig, 0, 1, [0, 1], 'lines', 'trace')
fig.show()

fig = go.Figure()
fig = reachtube_tree(traces_veri, None, fig, 0, 1, [0, 1], 'lines', 'trace')
fig.show()

# 3. Adding safety assertions

In [ ]:
from verse.scenario import Scenario
from tutorial_map import M1
scenario = Scenario()
scenario.config.init_seg_length = 5
scenario.set_map(M1())

In [ ]:
from enum import Enum, auto

class AgentMode(Enum):
    Normal = auto()
    Brake = auto()
    
class TrackMode(Enum):
    T0 = auto()
    
class State:
    x:float
    y:float
    theta:float
    v:float
    agent_mode:AgentMode 
    track_mode:TrackMode 

    def __init__(self, x, y, theta, v, agent_mode: AgentMode, track_mode: TrackMode):
        pass

Verse allow checking safety conditions while performing simulation and verification. The safety conditions in verse can be specified using Python assert statements in the decision logic. With the <code>any</code> and <code>all</code> functions, the user can also define safety conditions between different agents. In this example, we are going to add a safety condition that the distance between two agents in x direction should always be greater than or equan to 1.0m. The updated decision logic is shown below. 

In [ ]:
from typing import List
import copy
def decisionLogic(ego:State, others: List[State], track_map):
    output = copy.deepcopy(ego)
    if ego.agent_mode == AgentMode.Normal:
        if any( other.x-ego.x< 8 and other.x-ego.x>0 for other in others):
            output.agent_mode = AgentMode.Brake
    
    ### Adding safety assertions
    assert not any(other.x-ego.x<1.0 and other.x-ego.x>-1.0 for other in others), 'Seperation'
    ##########
    
    return output

We can then construct the two agents and the scenario exactly as previous section.

In [ ]:
from tutorial_agent import Agent1
car1 = Agent1('car1', file_name="dl_sec3.py")
car1.set_initial([[0,-0.5,0,2],[1,0.5,0,2]], (AgentMode.Normal, TrackMode.T0))
car2 = Agent1('car2', file_name="dl_sec3.py")
car2.set_initial([[15,-0.5,0,1],[16,0.5,0,1]], (AgentMode.Normal, TrackMode.T0))
scenario.add_agent(car1)
scenario.add_agent(car2)
from tutorial_sensor import DefaultSensor
scenario.set_sensor(DefaultSensor())

We can then simulate and verify the scenario and visualize the result. We can see from the result that this scenario is safe. 

In [ ]:
traces_simu = scenario.simulate(10, 0.01)
traces_veri = scenario.verify(10, 0.01)

In [ ]:
import plotly.graph_objects as go
from verse.plotter.plotter2D import *

fig = go.Figure()
fig = simulation_tree(traces_simu, None, fig, 0, 1, [0, 1], 'lines', 'trace')
fig.show()

fig = go.Figure()
fig = reachtube_tree(traces_veri, None, fig, 0, 1, [0, 1], 'lines', 'trace')
fig.show()

If we tweak the initial condition of car1 by increasing it's speed to 6m/s and redo verification, the safety condition is violated and the result is shown in the plot. 

In [ ]:
scenario.set_init_single('car1',[[0,-0.5,0,5],[1,0.5,0,5]], (AgentMode.Normal, TrackMode.T0))

traces_veri = scenario.verify(10, 0.01)

In [ ]:
import plotly.graph_objects as go
from verse.plotter.plotter2D import *

fig = go.Figure()
fig = reachtube_tree(traces_veri, None, fig, 0, 1, [0, 1], 'lines', 'trace')
fig.show()

# 4. Creating agent flow

In previous sections, we discussed about how to create the decision logic for agents in Verse, in section we will look into detail about how the agent flows are specified and how the pieces are combined together for an agent in Verse. 

The flow function define the continuous time evolution of agents' continuous states. It takes as input the initial states of the agent and the a time, and outputs the state of the agent from that initial states at that time. In Verse, the flow function is implemented by the <code>TC_simulate</code> function in the agent class. The <code>TC_simulate</code> takes as input a mode, a initial continuous states, a time bound, a simulation time step and the map. It will output simulation of agent dynamics at given mode starting from the initial continuous states, until the time bound with time step as an np array. Below shows an implementation of the <code>TC_simulate</code> function for the car agent. Note some detail helper functions for the car dynamics placed in <code>tutorial_utils</code> are not shown explicitly in the example below. 

In [ ]:
from tutorial_utils import car_dynamics, car_action_handler
import numpy as np 
from scipy.integrate import ode

def TC_simulate(self, mode: List[str], initialCondition, time_bound, time_step, track_map=None)->np.ndarray:
    time_bound = float(time_bound)
    number_points = int(np.ceil(time_bound/time_step))
    t = [round(i*time_step,10) for i in range(0,number_points)]

    init = initialCondition
    trace = [[0]+init]
    for i in range(len(t)):
        steering, a = car_action_handler(mode, init, track_map)
        r = ode(car_dynamics)    
        r.set_initial_value(init).set_f_params([steering, a])      
        res:np.ndarray = r.integrate(r.t + time_step)
        init = res.flatten().tolist()
        if init[3] < 0:
            init[3] = 0
        trace.append([t[i] + time_step] + init) 

    return np.array(trace)


With the <code>TC_simulate</code> function specified, we can now combine all parts together to construct a new car agent. An agent class for Verse should have three attributes: 1) <code>id</code>: an unique identifier for each agent in a scenario, 2) <code>controller</code>: a <code>ControllerIR</code> object, which is an intermediate representation of decisionLogic in Verse. It can be constructed automatically by passing in the decision logic code strings described in previous sections, and 3) <code>TC_simulate</code> function. In this example we will create the agent class manually. However, the agent class can also be inherited from the <code>BaseAgent</code> class with proper arguments to automatically having all required fields.

In [ ]:
class Agent2:
    pass